In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import sys

In [2]:
!ls ../

config	deepseadream  Makefile	README.md	  scripts
data	docker	      notebook	requirements.txt  tests


In [3]:
PATH = "../data/data/train_fish/"
ALB = "ALB"
BET = "BET"
DOL = "DOL"
LAG = "LAG"
NoF = "NoF"
OTHER = "OTHER"
SHARK = "SHARK"
YFT = "YFT"

In [4]:
import os
for path, di, file  in os.walk(PATH):
    CLASS = di
    break
NUM_CLASSES = len(CLASS)
IMAGE_SIZE = 256
IMAGE_PIXELS = IMAGE_SIZE*IMAGE_SIZE*3
learning_rate = 1e-4
max_steps = 200
BATCH_SIZE = 100

In [5]:
# これらコードは、http://kivantium.hateblo.jp/entry/2015/11/18/233834から得られる
def inference(images_placeholder,keep_prob):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)
    def bias_variable(shape):
        initial = tf.constant(0.1,shape=shape)
        return tf.Variable(initial)
    def conv2d(x,W):
        return tf.nn.conv2d(x,W,[1,1,1,1],padding='SAME')
    def max_pool_2x2(x):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    


    x_image = tf.reshape(images_placeholder,[-1,256,256,3])

    

    with tf.name_scope('conv1') as scope:
        W_conv1 = weight_variable([5,5,3,32])
#         W_conv1 = weight_variable([5,5,1,32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)

    with tf.name_scope('pool1') as scope:
        h_pool1 = max_pool_2x2(h_conv1)
        
#         h_pool1 = tf.reduce_max(h_conv1,reduction_indices=[3],keep_dims=True)

    with tf.name_scope('conv2') as scope:
        W_conv2 = weight_variable([5,5,32,64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
        # h_conv2 has (?,128, 128, 64) shape
    with tf.name_scope('pool2') as scope:
        h_pool2 = max_pool_2x2(h_conv2)
        # h_pool2 has (?, 64, 64, 64) shape
    #全結合
    with tf.name_scope('zenketsu1') as scope:
        W_fc1 = weight_variable([64*64*64,1024])
#         W_fc1 = weight_variable([5*5*32,1024])
        b_fc1 = bias_variable([1024])
#         h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64]) #3136 defined here
        h_pool2_flat = tf.reshape(h_pool2,[-1,64*64*64]) #3136 defined here
        
        # h_pool2_flatは二回poolされているから4で割った値を取るのか？この場合256/4
        
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
        #dropout
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
        
    with tf.name_scope('zenketsu2') as scope:
        W_fc2 = weight_variable([1024,NUM_CLASSES])
        b_fc2 = bias_variable([NUM_CLASSES])
    
    with tf.name_scope('softmax') as scope:
        y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
    
    return y_conv

In [6]:
def loss(logits,labels):
    cross_entropy = -tf.reduce_sum(labels*tf.log(logits))
#     tf.scalar_summary("cross_entropy",cross_entropy)
    return cross_entropy

In [7]:
def training(loss,learning_rate):
    train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)
    return train_step

In [8]:
# file load
def load_files(path):
    data = []
    save_pa = []
    for pa,_,_ in os.walk(path):
        if pa =='../data/data/train_fish/':
            continue
        save_pa.append(pa)
    for i_c, cat in enumerate(save_pa):
        for _,_,files in os.walk(cat):
            for file in files:
                label = np.zeros(8)
                label[i_c] = 1
                filepath = os.path.join(cat,file)
                data.append([filepath,label])
    return data
DATA = load_files(PATH)
import csv
with open("../data/meta.csv",'w') as f:
    writer = csv.writer(f,lineterminator='\n')
    writer.writerow(DATA)

In [9]:
DATA[0]

['../data/data/train_fish/NoF/img_04395.jpg',
 array([1., 0., 0., 0., 0., 0., 0., 0.])]

In [10]:
len(DATA)

3777

In [11]:
# DATA = [(d[1],d[0]) for d in DATA]
import random
random.shuffle(DATA)
num_train = int(len(DATA) * 0.8)
train = DATA[:num_train]
test = DATA[num_train:]

In [12]:
def train_batch(data_list):
    labels,paths = [],[]
    for data in data_list:
        labels.append(data[0])
        paths.append(data[1])
    queue = tf.train.slice_input_producer([labels,paths])
    label = queue[0]


    jpeg = tf.read_file(queue[1])
    image = tf.image.decode_jpeg(jpeg,channels=3)
    image = tf.image.resize_images(image,[256,256])

    image = tf.image.per_image_standardization(image)

    return tf.train.shuffle_batch([image,label],batch_size=32,
                                  capacity = len(data_list) * 2 + 3* 32,
                                 min_after_dequeue = len(data_list) *2)

In [13]:
def test_batch(data_list):
    labels,paths = [],[]
    for data in data_list:
        labels.append(data[0])
        paths.append(data[1])
    queue = tf.train.slice_input_producer([labels,paths])
    label = queue[0]
    jpeg = tf.read_file(queue[1])
    image = tf.image.decode_jpeg(jpeg,channels=3)
    image = tf.image.resize_images(image,[256,256])

    image = tf.image.per_image_standardization(image)

    return tf.train.batch([image,label],len(data_list))

In [14]:
def accuracy(logits,labels):
    correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    return accuracy
def split_image_and_label(data_list,LESS=False):
    '''
    data_list should be [[data,label]...]
    '''
    data = []
    labels = []
    if LESS:
        i=0
        for value,label in data_list:
            i+=1
            data.append(value)
            labels.append(label)
            if i == 1000:
                break
        return data,labels
    else:
        for value,label in data_list:
            data.append(value)
            labels.append(label)
        return data, labels

In [15]:
# 以下のセルにあるtrain, testは実データなので、読み込みが必要。
def convert_image_path_to_data(path):
    jpeg_r = tf.read_file(path)
    image = tf.image.decode_jpeg(jpeg_r,channels=3)
    image = tf.image.resize_images(image,[256,256])
    image = tf.image.per_image_standardization(image)

    return image


In [16]:
# opnecvによる読み込み
import cv2 
def convert_image_path_to_data_cv2(path,i):
    try:
#         img2 = cv2.imread(path,0) # 白黒画像で読み込み
        img2 = cv2.imread(path)

        img2 = cv2.resize(img2,dsize=(256,256),interpolation = cv2.INTER_CUBIC)
        np_image_data = np.asarray(img2)
        np_final = np.expand_dims(np_image_data,axis=0)
        np_final = flatten(np_final)
        if i % 1000 == 0:
            print(i)
        #1000 images / 15s
        return np_final
    except:
        print(path)
        print(i)
        return np.zeros([256,256])

In [17]:
def flatten(image):
    image = image.flatten().astype(np.float32)/255.0
    return image

In [18]:
print(IMAGE_PIXELS)

196608


In [20]:
len(train_image)

NameError: name 'train_image' is not defined

In [21]:
train[10]

['../data/data/train_fish/ALB/img_05902.jpg',
 array([0., 0., 1., 0., 0., 0., 0., 0.])]

In [ ]:
with tf.Graph().as_default():
    images_placeholder = tf.placeholder("float",shape=(None,IMAGE_PIXELS))
    labels_placeholder = tf.placeholder("float",shape=(None,NUM_CLASSES))
    keep_prob = tf.placeholder("float")
    
    logits = inference(images_placeholder,keep_prob)
    loss_value = loss(logits,labels_placeholder)
    train_op = training(loss_value,learning_rate)
    acc = accuracy(logits,labels_placeholder)

    #for debugging True
    train_image, train_label = split_image_and_label(train,False)
    test_image,test_label = split_image_and_label(test,False) 
    print(len(train_image))
    print(len(test_image))
    
    train_image = [convert_image_path_to_data_cv2(path,i) for i,path in enumerate(train_image)]
    test_image = [convert_image_path_to_data_cv2(path,i) for i,path in enumerate(test_image)]
    #takes 20 mins
    print("success converting")
    
    
    # 保存の準備
    saver = tf.train.Saver()
    # Sessionの作成
    sess = tf.Session()
    # 変数の初期化
    sess.run(tf.initialize_all_variables())

    # TensorBoardで表示する値の設定
#     summary_op = tf.merge_all_summaries()
#     summary_writer = tf.train.SummaryWriter(FLAGS.train_dir, sess.graph_def)

    # 訓練の実行
    for step in range(max_steps):
        for i in range(int(len(train_image)/BATCH_SIZE)):
            # batch_size分の画像に対して訓練の実行
            batch = BATCH_SIZE*i
            # feed_dictでplaceholderに入れるデータを指定する
            sess.run(train_op, feed_dict={
              images_placeholder: train_image[batch:batch+BATCH_SIZE],
              labels_placeholder: train_label[batch:batch+BATCH_SIZE],
              keep_prob: 0.5})

        # 1 step終わるたびに精度を計算する
        train_accuracy = sess.run(acc, feed_dict={
            images_placeholder: train_image,
            labels_placeholder: train_label,
            keep_prob: 1.0})
        print("step %d, training accuracy %g"%(step, train_accuracy))

#         # 1 step終わるたびにTensorBoardに表示する値を追加する
#         summary_str = sess.run(summary_op, feed_dict={
#             images_placeholder: train_image,
#             labels_placeholder: train_label,
#             keep_prob: 1.0})
#         summary_writer.add_summary(summary_str, step)

# 訓練が終了したらテストデータに対する精度を表示
print("test accuracy %g"%sess.run(acc, feed_dict={
    images_placeholder: test_image,
    labels_placeholder: test_label,
    keep_prob: 1.0}))

# 最終的なモデルを保存
save_path = saver.save(sess, "model.ckpt")

3021
756
0
1000
2000
3000
0
success converting
Instructions for updating:
Use `tf.global_variables_initializer` instead.
